In [13]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('data/Hotel_Reviews_Cleaned.csv')

df.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Cleaned_Positive_Reviews,Cleaned_Negative_Reviews
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,park outside hotel beautiful,angry made post available via possible sites u...
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,real complaints hotel great great location sur...,negative
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968,location good staff ok cute hotel breakfast ra...,rooms nice elderly bit difficult rooms two sto...
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968,great location nice surroundings bar restauran...,room dirty afraid walk barefoot floor looked c...
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968,amazing location building romantic setting,booked company line showed pictures room thoug...


In [3]:
# import re
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize

# def preprocess_text(text):
#   text = re.sub(r'\d+', '', text)  # Remove numbers
#   text = text.lower()  # Convert to lowercase
#   text = re.sub(r'\s+', ' ', text)  # Remove extra whitespaces
#   text = ' '.join([word for word in word_tokenize(text) if word.isalpha() and word not in stopwords.words('english')])
#   return text

# df['Cleaned_Positive_Reviews'] = df['Positive_Review'].apply(preprocess_text)
# df['Cleaned_Negative_Reviews'] = df['Negative_Review'].apply(preprocess_text)

In [4]:
# df.to_csv('data/Hotel_Reviews_Cleaned.csv', index=False)

In [5]:
df['Cleaned_Positive_Reviews'].fillna('', inplace=True)


In [6]:
df['Cleaned_Negative_Reviews'].fillna('', inplace=True)

In [7]:
positive_hotel_reviews = df.groupby('Hotel_Name')['Cleaned_Positive_Reviews'].apply(lambda x: ' '.join(x)).reset_index()

In [8]:
negative_hotel_reviews = df.groupby('Hotel_Name')['Cleaned_Negative_Reviews'].apply(lambda x: ' '.join(x)).reset_index()

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

def extract_top_keywords(text, n=5):
  vectorizer = CountVectorizer()
  X = vectorizer.fit_transform([text])
  terms = vectorizer.get_feature_names_out()
  keywords = [terms[i] for i in X.toarray().argsort()[0][-n:]][::-1]
  return keywords

positive_hotel_reviews['Top_Positive_Keywords'] = positive_hotel_reviews['Cleaned_Positive_Reviews'].apply(extract_top_keywords)
negative_hotel_reviews['Top_Negative_Keywords'] = negative_hotel_reviews['Cleaned_Negative_Reviews'].apply(extract_top_keywords)

In [66]:
random_hotels = df['Hotel_Name'].sample(n=10).values

print("List of Hotels:\n")
print(random_hotels)
print("*"*50)

selected_hotel = input("\nEnter the name of the hotel: ")
print(f"\nYou selected {selected_hotel}")
print("*"*50)

if selected_hotel in positive_hotel_reviews['Hotel_Name'].values:
    top_positive_keywords = positive_hotel_reviews[positive_hotel_reviews['Hotel_Name'] == selected_hotel]['Top_Positive_Keywords'].values[0]
    top_negative_keywords = negative_hotel_reviews[negative_hotel_reviews['Hotel_Name'] == selected_hotel]['Top_Negative_Keywords'].values[0]

    keywords = list(set(top_positive_keywords + top_negative_keywords))

    print(f"\nTop keywords for the hotel '{selected_hotel}':\n")
    print(keywords)

    selected_keyword = input("\nEnter a keyword: ")
    print(f"\nYou selected {selected_keyword}")
    print("*"*50)

    # Find the index of the selected hotel
    selected_hotel_indexes_list = df.index[df['Hotel_Name'] == selected_hotel].to_list()

    # Find reviews similar to the selected keyword within the specified hotel
    similar_positive_reviews = []
    similar_negative_reviews = []

    if selected_keyword in keywords:
        for i in selected_hotel_indexes_list:
            pos_review = df.iloc[i]['Cleaned_Positive_Reviews']
            neg_review = df.iloc[i]['Cleaned_Negative_Reviews']
            if selected_keyword in df.iloc[i]['Cleaned_Positive_Reviews'] and selected_keyword in df.iloc[i]['Cleaned_Negative_Reviews']:
                similar_positive_reviews.append(df.iloc[i]['Positive_Review'])
                similar_negative_reviews.append(df.iloc[i]['Negative_Review'])
            elif selected_keyword in df.iloc[i]['Cleaned_Positive_Reviews']:
                similar_positive_reviews.append(df.iloc[i]['Positive_Review'])
            elif selected_keyword in df.iloc[i]['Cleaned_Negative_Reviews']:
                similar_negative_reviews.append(df.iloc[i]['Negative_Review'])
            else:
                continue
        print(f"\n {len(similar_positive_reviews)}/{len(selected_hotel_indexes_list)} positive reviews contain the keyword '{selected_keyword}' for the hotel '{selected_hotel}':\n")
        for review in similar_positive_reviews:
            print("-", review)

        print(f"\n {len(similar_negative_reviews)}/{len(selected_hotel_indexes_list)} negative reviews contain the keyword '{selected_keyword}' for the hotel '{selected_hotel}':\n")
        for review in similar_negative_reviews:
            print("-", review)
    else:
        print("Invalid keyword.")
else:
    print("Invalid hotel name.")

List of Hotels:

['Fletcher Hotel Amsterdam' 'Montagu Place Hotel'
 'Holiday Inn London Bloomsbury' 'Olivia Plaza Hotel'
 'DoubleTree by Hilton London Docklands Riverside'
 'Intercontinental London The O2' 'Golden Tulip Amsterdam Riverside'
 'Park Hotel' 'Best Western Hotel Montmartre Sacr Coeur'
 'Catalonia Ramblas 4 Sup']
**************************************************

You selected Olivia Plaza Hotel
**************************************************

Top keywords for the hotel 'Olivia Plaza Hotel':

['great', 'location', 'breakfast', 'hotel', 'nothing', 'staff', 'room', 'negative']

You selected room
**************************************************

 262/773 positive reviews contain the keyword 'room' for the hotel 'Olivia Plaza Hotel':

-  We were upgraded to a penthouse room with a spectacular view huge soaker tub lovely bathroom overall Amazing location just steps from the Ramblas and an easy walk to the Gaud sites or the Barri Gotic We had dinner at the restaurant and thou

### TF-IDF

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF Vectorization for each hotel
hotel_tfidf_matrices = {}
for hotel_name, group in df.groupby('Hotel_Name'):
  positive_vectorizer = TfidfVectorizer()
  negative_vectorizer = TfidfVectorizer()
  tfidf_matrix_positive = positive_vectorizer.fit_transform(group['Cleaned_Positive_Reviews'])
  tfidf_matrix_negative = negative_vectorizer.fit_transform(group['Cleaned_Negative_Reviews'])
  hotel_tfidf_matrices[hotel_name] = {'positive_vectorizer': positive_vectorizer, 'negative_vectorizer': negative_vectorizer , 'tfidf_matrix_positive': tfidf_matrix_positive, 'tfidf_matrix_negative': tfidf_matrix_negative}

# Function to extract top keywords based on TF-IDF for a specific hotel
def extract_top_keywords_for_hotel(document, hotel_name, top_n=5, positive=True):
  positive_vectorizer = hotel_tfidf_matrices[hotel_name]['positive_vectorizer']
  negative_vectorizer = hotel_tfidf_matrices[hotel_name]['negative_vectorizer']
  
  if positive:
    tfidf_scores = positive_vectorizer.transform([document])
    feature_names = positive_vectorizer.get_feature_names_out()
    sorted_indices = tfidf_scores.indices[np.argsort(tfidf_scores.data)][::-1][:top_n]
    keywords = [feature_names[idx] for idx in sorted_indices]
    return keywords
  else:
    tfidf_scores = negative_vectorizer.transform([document])
    feature_names = negative_vectorizer.get_feature_names_out()
    sorted_indices = tfidf_scores.indices[np.argsort(tfidf_scores.data)][::-1][:top_n]
    keywords = [feature_names[idx] for idx in sorted_indices]
    return keywords

In [47]:
positive_hotel_reviews['Top_Positive_Keywords_TFIDF'] = positive_hotel_reviews.apply(lambda x: extract_top_keywords_for_hotel(x['Cleaned_Positive_Reviews'], x['Hotel_Name']), axis=1)
negative_hotel_reviews['Top_Negative_Keywords_TFIDF'] = negative_hotel_reviews.apply(lambda x: extract_top_keywords_for_hotel(x['Cleaned_Negative_Reviews'], x['Hotel_Name'], positive=False), axis=1)

### KeyBERT

In [51]:
from keybert import KeyBERT

In [53]:
kw_model = KeyBERT('all-mpnet-base-v2')

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [58]:
def extract_top_keywords_keybert(document, top_n=5):
  keywords = kw_model.extract_keywords(document, keyphrase_ngram_range=(1, 1), stop_words='english', top_n=top_n)
  return [keyword[0] for keyword in keywords]

In [59]:
positive_hotel_reviews['Top_Positive_Keywords_KeyBERT'] = positive_hotel_reviews['Cleaned_Positive_Reviews'].apply(extract_top_keywords_keybert)
negative_hotel_reviews['Top_Negative_Keywords_KeyBERT'] = negative_hotel_reviews['Cleaned_Negative_Reviews'].apply(extract_top_keywords_keybert)

KeyboardInterrupt: 

In [64]:
negative_hotel_reviews.head(15).to_excel('data/negative_hotel_reviews.xlsx', index=False)